In [45]:
import os 
from dotenv import load_dotenv 
import re
import google.generativeai as genai

load_dotenv()
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
from diffusers import StableDiffusionPipeline

WARNING[XFORMERS]: xFormers can't load C++/CUDA extensions. xFormers was built for:
    PyTorch 2.3.0+cu121 with CUDA 1201 (you have 2.3.1+cu118)
    Python  3.10.11 (you have 3.10.14)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
  Set XFORMERS_MORE_DETAILS=1 for more details
A matching Triton is not available, some optimizations will not be enabled
Traceback (most recent call last):
  File "c:\Users\dhanu\.conda\envs\lm_python\lib\site-packages\xformers\__init__.py", line 55, in _is_triton_available
    from xformers.triton.softmax import softmax as triton_softmax  # noqa
  File "c:\Users\dhanu\.conda\envs\lm_python\lib\site-packages\xformers\triton\softmax.py", line 11, in <module>
    import triton
ModuleNotFoundError: No module named 'triton'


In [12]:
GOOGLE_API_KEY = ""
genai.configure(api_key=GOOGLE_API_KEY)

In [38]:
class Chatbot:
    def __init__(self, system):
        self.system = system
        self.messages = []
        if self.system:
            self.messages.append({"role": "system", "content": system})

    def __call__(self, message):
        self.messages.append({"role": "user", "content": message})
        result = self.execute()
        self.messages.append({"role": "assistant", "content": result})
        return result

    def execute(self):
        prompt = "\n".join([f'{msg["role"]}:{msg["content"]}' for msg in self.messages])
        model = genai.GenerativeModel("gemini-1.5-flash")
        raw_response = model.generate_content(prompt)
        # Extract the text content directly from the candidates list
        result_text = raw_response.candidates[0].content.parts[0].text
        return result_text

In [39]:




prompt = """ 
You are a fitness assistant. You help users with workout plans, dietary advice, and motivational quotes.
Your available actions are:
generate_workout:
e.g. generate_workout: Beginner 
Generates a workout plan based on the user's fitness level.
suggest_meal:
e.g. suggest_meal: Low-carb breakfast 
Suggests a meal plan based on the user's dietary preferences.
motivational_quotes:
e.g. motivational_quotes:
Returns a motivational quote to inspire the user.

Example session:
Question: Can you help me with a beginner workout plan?
Thought: I should generate a workout plan.
Action: generate_workout: Beginner

PAUSE 

Observation: Here is a beginner workout plan: 10 pushups, 20 squats, 30 seconds plank.
Answer: I suggest starting with 10 pushups, 20 squats, and 30 seconds plank for your home workout.
""".strip()

In [40]:

# Action regex 
action_re = re.compile(r'^Action:\s*(\w+)\s*:\s*(.+)$')

In [41]:



# Action functions using Gemini API 
def generate_workout(level):
    response = genai.GenerativeModel("gemini-1.5-flash").generate_content(f"Generate a workout plan for a {level} fitness level")
    return response.candidates[0].content.parts[0].text 

def suggest_meal(preferences):
    response = genai.GenerativeModel("gemini-1.5-flash").generate_content(f"Suggest a meal plan with {preferences}")
    return response.candidates[0].content.parts[0].text

def motivational_quotes():
    response = genai.GenerativeModel("gemini-1.5-flash").generate_content("Give me a motivational quote.")
    return response.candidates[0].content.parts[0].text

# Mapping actions to functions
known_actions = {
    "generate_workout": generate_workout,
    "suggest_meal": suggest_meal,
    "motivational_quotes": motivational_quotes
}

In [37]:
# Mapping actions to functions
known_actions = {
    "generate_workout": generate_workout,
    "suggest_meal": suggest_meal,
    "motivational_quotes": motivational_quotes
}

In [42]:


# Query function
def query(question, max_turns=5):
    i = 0
    bot = Chatbot(prompt)
    next_prompt = question
    while i < max_turns:
        i += 1
        result = bot(next_prompt)
        print(result)
        actions = [action_re.match(a) for a in result.split('\n') if action_re.match(a)]

        if actions:
            action, action_input = actions[0].groups()
            if action not in known_actions:
                raise Exception(f"Unknown action: {action}: {action_input}")
            print(f"-- running {action} {action_input}")
            observation = known_actions[action](action_input.strip())
            print(f"Observation: {observation}")
            next_prompt = f"Answer: {observation}"
        else:
            return 



In [43]:


query("Can you help me with a beginner workout plan?")


Thought: I should generate a workout plan.
Action: generate_workout: Beginner 

-- running generate_workout Beginner 
Observation: ## Beginner Workout Plan: Get Started with Fitness!

**Goal:** Build a foundation of fitness with this plan that focuses on strength, cardio, and flexibility.

**Frequency:** 3-4 times per week, with at least one rest day between workouts.

**Warm-up:** 5-10 minutes of light cardio, like marching in place, jumping jacks, or arm circles.

**Cool-down:** 5-10 minutes of stretching. Hold each stretch for 20-30 seconds.

**Workout:**

**Day 1: Strength Training (Upper Body)**

* **Push-ups:** 3 sets of 8-12 reps. (Modify by doing push-ups against a wall or on your knees).
* **Dumbbell Rows:** 3 sets of 8-12 reps. (Use light dumbbells or resistance bands).
* **Overhead Press:** 3 sets of 8-12 reps. (Use light dumbbells).
* **Bicep Curls:** 3 sets of 10-15 reps. (Use light dumbbells).
* **Tricep Extensions:** 3 sets of 10-15 reps. (Use light dumbbells or resistan